In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [2]:
df = pd.read_csv("data/us_estdata.csv")
df.head()

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,EDU_ORIG,MED_ORIG,ENT_ORIG,FOD_ORIG,SRV_ORIG,PUB_ORIG,JOBS_EDU_NOHS_ORIG,JOBS_EDU_HS_ORIG,JOBS_EDU_NOBACH_ORIG,JOBS_EDU_BACH_ORIG
0,0,P,2018GQ0103142,2,1,902,1,34,1013097,27,...,38315.0,54857.0,4668.0,25243.0,13545.0,12088.0,44574.0,75277.0,96405.0,139711.0
1,1,P,2018HU1096416,9,1,10200,4,53,1013097,341,...,7051.0,9608.0,2445.0,7125.0,2552.0,3823.0,6449.0,14893.0,18922.0,14092.0
2,2,P,2018HU0306186,7,2,1400,3,48,1013097,43,...,2964.0,3722.0,179.0,3216.0,886.0,688.0,4016.0,6496.0,7050.0,4215.0
3,3,P,2018HU0641097,7,1,1200,3,48,1013097,24,...,3558.0,3012.0,97.0,2808.0,898.0,988.0,3882.0,7306.0,7491.0,4446.0
4,4,P,2018HU1246748,5,1,7300,3,12,1013097,75,...,18126.0,21139.0,2060.0,17590.0,6016.0,35219.0,15006.0,31118.0,38494.0,34392.0


In [3]:
# clean up the database (Biogeme Database can only have numerical values)
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [4]:
df["CHOSEN"].value_counts()

102500     106
5500100     85
5310200     84
5500700     74
1700202     73
          ... 
4804619      7
2701404      7
5541001      6
2601703      5
3500806      4
Name: CHOSEN, Length: 2336, dtype: int64

In [5]:
# defining the chosen alterantive for each person explicitly (0 to 35, corresponding to staying and moving to one of the many PUMAs)
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 101): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])
df["CHOSEN"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [6]:
df["CHOSEN"].value_counts()

0    55261
1     8011
Name: CHOSEN, dtype: int64

In [7]:
df["IN_COLLEGE"] = np.where((df["SCHG"] == 15) | (df["SCHG"] == 16), 1, 0)
df["IN_COLLEGE"]

0        0
1        0
2        0
3        0
4        0
        ..
63267    0
63268    0
63269    0
63270    0
63271    0
Name: IN_COLLEGE, Length: 63272, dtype: int32

In [8]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [9]:
df["AGE_18_22"] = np.where(df["AGEP"] <= 22, 1, 0)
df["AGE_23_29"] = np.where((df["AGEP"] >= 23) & (df["AGEP"] <= 29), 1, 0)
df["AGE_30_39"] = np.where((df["AGEP"] >= 30) & (df["AGEP"] <= 39), 1, 0)
df["AGE_40_49"] = np.where((df["AGEP"] >= 40) & (df["AGEP"] <= 49), 1, 0)
df["AGE_50_64"] = np.where((df["AGEP"] >= 50) & (df["AGEP"] <= 64), 1, 0)

In [10]:
df["EDU_LESS_HIGH"] = np.where(df["SCHL"] <= 15, 1, 0)
df["EDU_HIGHONLY"] = np.where((df["SCHL"] >= 16) & (df["SCHL"] <= 17), 1, 0)
df["EDU_SOMECOLLEGE"] = np.where((df["SCHL"] >= 18) & (df["SCHL"] <= 20), 1, 0)
df["EDU_COLLEGE"] = np.where(df["SCHL"] >= 21, 1, 0)
df["EDU_NOCOLLEGE"] = np.where(df["EDU_COLLEGE"] == 0, 1, 0)

In [11]:
df["WOMAN_CHILD"] = np.where((df["PAOC"] >= 1) & (df["PAOC"] <= 3), 1, 0)
df["UNEMPLOYED"] = np.where(df["ESR"] == 3, 1, 0)

In [12]:
df["MALE"] = np.where(df["SEX"] == 1, 1, 0)
df["FEMALE"] = np.where(df["SEX"] == 0, 1, 0)

In [13]:
df["MARRIED"] = np.where(df["MAR"] == 1, 1, 0)

In [17]:
# df["child_old"] = np.where(df["child"] == df["REC_CHILD"], 0, df["child"])
# df["child_old"].value_counts()

In [15]:
df["REC_NO_MAR"] = np.where((df["MARHD"] == 1) | (df["MARHW"] == 1), 1, 0)
df["REC_NO_MAR"].value_counts()

0    62389
1      883
Name: REC_NO_MAR, dtype: int64

In [16]:
df["MARHM_new"] = np.where(df["MARHM"] == 2, 0, df["MARHM"])
df["MARHM_new"].value_counts()

0.0    62261
1.0     1011
Name: MARHM_new, dtype: int64

In [17]:
df["MARRIED"].value_counts()

1    34051
0    29221
Name: MARRIED, dtype: int64

In [18]:
df["married_old"] = np.where((df["MARHM"] == df["MARRIED"]), 0, df["MARRIED"])
df["married_old"].value_counts()

1    33059
0    30213
Name: married_old, dtype: int64

In [19]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df)

In [20]:
# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [73]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not the coefficinet should be fixed to initial value value)
c_destchoice_emp=Beta('c_destchoice_emp', 0, None, None, 0)
c_destchoice_dist=Beta('c_destchoice_dist', 0, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', 0, None, None, 0)
c_destchoice_time=Beta('c_destchoice_time', 0, None, None, 0)
c_destchoice_unemp = Beta("c_destchoice_unemp", 0, None, None, 0)
c_destchoice_hhinc = Beta("c_destchoice_hhinc", 0, None, None, 0)
c_destchoice_internal = Beta("c_destchoice_internal", 0, None, None, 0)
c_destchoice_urban = Beta("c_destchoice_urban", 0, None, None, 0)
c_destchoice_huprice = Beta("c_destchoice_huprice", 0, None, None, 0)
c_destchoice_hurent = Beta("c_destchoice_hurent", 0, None, None, 0)
c_destchoice_vacancy = Beta("c_destchoice_vacancy", 0, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 0, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 0, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
c_destchoice_unemp = Beta("c_destchoice_unemp", 0, None, None, 0)
c_destchoice_pctbach = Beta("c_destchoice_pctbach", 0, None, None, 0)
c_destchoice_pctnobach = Beta("c_destchoice_pctnobach", 0, None, None, 0)
c_destchoice_pctownind = Beta("c_destchoice_pctownind", 0, None, None, 0)
c_destchoice_entscore = Beta("c_destchoice_entscore", 0, None, None, 0)

In [74]:
c_move = Beta("c_move", 0, None, None, 0)

In [75]:
# Staying Choice Parameters to be Estimated
c_stay_married = Beta("c_stay_married", 0, None, None, 0)
c_stay_income = Beta("c_stay_income", 0, None, None, 0)
c_stay_origin_unemp_rate = Beta("c_stay_origin_unemp", 0, None, None, 0)
c_stay_age_18_22 = Beta("c_stay_age_18_22", 0, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", 0, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", 0, None, None, 0)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", 0, None, None, 0)
c_stay_child = Beta("c_stay_child", 0, None, None, 0)
c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
c_stay = Beta("c_stay", 0, None, None, 0)
c_stay_hhinc_orig = Beta("c_stay_hhinc_orig", 0, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
c_stay_rent = Beta("c_stay_rent", 0, None, None, 0)
c_stay_hurent = Beta("c_stay_hurent", 0, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
c_stay_college = Beta("c_stay_college", 0, None, None, 0)

In [76]:
c_stay_rec_child = Beta("c_stay_rec_child", 0, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", 0, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", 0, None, None, 0)

In [77]:
# defining the staying utility function
V0 = c_stay + c_stay_married * MARRIED + c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_income * PINCP

# c_stay_married * MARRIED + c_stay_income * PINCP + c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_edu_nohigh * EDU_LESS_HIGH + c_stay_edu_somecollege * EDU_SOMECOLLEGE + c_stay_edu_college * EDU_COLLEGE + c_stay_child * child + c_stay + c_stay_hhinc_orig * HH_MED_INC_ORIG + c_stay_foreign * FOREIGN + c_stay_hurent * HH_MED_RENT_ORIG + c_stay_dens * DENS_ORIG + c_stay_origin_unemp_rate * UNEMP_ORIG + c_stay_college * IN_COLLEGE

In [78]:
df["DENS_ORIG"]

0        2683.772098
1         277.144696
2         164.757333
3         140.816820
4         438.644423
            ...     
63267    3629.639047
63268     607.255300
63269     139.531553
63270    1502.962035
63271     354.855895
Name: DENS_ORIG, Length: 63272, dtype: float64

In [94]:
migpuma_acs_data = pd.read_csv("data/ACS_MIGPUMA_2018.csv").drop("Geo_FIPS", axis=1).set_index("MIGPUMA")

In [104]:
df[df["DENS_ORIG"] == 0]["ORIGIN"].value_counts()

1304007    139
5500104     79
4500606     69
5151001     38
Name: ORIGIN, dtype: int64

In [103]:
migpuma_acs_data.loc["5151001***"]

Geo_LOGRECNO                                                             189.000000
Geo_STATE                                                                102.000000
Geo_PUMA5                                                             102179.000000
Total Population                                                      252684.000000
Population Density (Per Sq. Mile)                                        201.743046
                                                                          ...      
Civilian Population 18 Years and Over Nonveteran 65 Years and Over     39469.000000
Total Population Native Born                                          233028.000000
Total Population Foreign Born                                          19656.000000
Total Population Foreign Born Naturalized Citizen                       9342.000000
Total Population Foreign Born Not a Citizen                            10314.000000
Name: 5151001***, Length: 155, dtype: float64

In [101]:
df.iloc[155, 2150:]

JOBS_EDU_BACH_ORIG          0.0
CHOSEN_PUMA           5151089.0
IN_COLLEGE                  0.0
AGE_18_34                   0.0
AGE_35_64                   0.0
AGE_OVER_65                 1.0
FOREIGN                     0.0
AGE_18_22                   0.0
AGE_23_29                   0.0
AGE_30_39                   0.0
AGE_40_49                   0.0
AGE_50_64                   0.0
EDU_LESS_HIGH               0.0
EDU_HIGHONLY                0.0
EDU_SOMECOLLEGE             0.0
EDU_COLLEGE                 1.0
EDU_NOCOLLEGE               0.0
WOMAN_CHILD                 0.0
UNEMPLOYED                  0.0
MALE                        1.0
FEMALE                      0.0
MARRIED                     1.0
filler                      0.0
REC_NO_MAR                  0.0
MARHM_new                   0.0
married_old                 1.0
Name: 155, dtype: float64

In [79]:
# defining the utility functions for each of the moving PUMA alternatives
# defined using the exec to parse a string to save space
# can also use a loop to print out the statements and then copy/paste them to run
# can also just write each one manually
for i in range(100):
    num = i + 1
    initialization = "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_urban * ALT{0}_DENS".format(num)
    exec(initialization)
print(V100)

# full model specification (takes a bit longer to run):
# V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_hhinc * int(puma_acs_data.loc['{1}', 'Median Household Income (In 2019 Inflation Adjusted Dollars)']) + c_destchoice_urban / DENS_ORIG * (puma_acs_data.loc['{1}', 'Population Density (Per Sq. Mile)'] - DENS_ORIG) + c_destchoice_hurent * int(puma_acs_data.loc['{1}', 'Median Gross Rent']) + c_destchoice_college * IN_COLLEGE * int(puma_acs_data.loc['{1}', 'COLLEGE']) + c_destchoice_vacancy * float(puma_acs_data.loc['{1}', 'VACANCY_PCT']) + c_destchoice_foreign * FOREIGN * int(puma_acs_data.loc['{1}', 'Total Population Foreign Born']) / ALT{0}_POP + c_destchoice_age_18_34 * AGE_18_34 * int(puma_acs_data.loc['{1}', 'Total Population 18 to 34 Years']) / ALT{0}_POP + c_destchoice_age_35_64 *AGE_35_64 * int(puma_acs_data.loc['{1}', 'Total Population 35 to 64 Years']) / ALT{0}_POP + c_destchoice_age_over_65 * AGE_OVER_65 * int(puma_acs_data.loc['{1}', 'Total Population 65 and Over']) / ALT{0}_POP + c_destchoice_pctnobach * EDU_NOCOLLEGE * int(puma_lodes_data.loc['{1}', ['JOBS_EDU_HS', 'JOBS_EDU_NOBACH', 'JOBS_EDU_NOHS']].sum()) / ALT{0}_EMP  + c_destchoice_pctbach * EDU_COLLEGE * int(puma_lodes_data.loc['{1}', 'JOBS_EDU_BACH']) / ALT{0}_EMP + c_destchoice_unemp * float(puma_acs_data.loc['{1}', 'UNEMP']) + c_destchoice_entscore * ALT{0}_ENT / ALT{0}_EMP / AGEP

# for the fields already in the Biogeme db.Database, can explicitly refer to them; also used a few references to other databases using .loc and fields in the Biogeme database

(((log(ALT100_POP) + (c_destchoice_dist(0) * ALT100_DIST)) + (c_destchoice_logdist(0) * log((ALT100_DIST + `1`)))) + (c_destchoice_urban(0) * ALT100_DENS))


In [80]:
utilities = {}
for i in range(101):
    init = "utilities[{0}] = V{0}".format(i)
    exec(init)

In [81]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()

In [82]:
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-05-17]\n[16:31:55] < General >   Remove 1561 unused variables from the database as only 615 are used.\n[16:31:56] < Detailed >  It is suggested to scale the following variables.\n[16:31:56] < Detailed >  Multiply ALT42_COLLEGE by\t1e-05 because the largest (abs) value is\t59710.0\n[16:31:56] < Detailed >  Multiply ALT2_DENS by\t1e-05 because the largest (abs) value is\t102846.33928097\n[16:31:56] < Detailed >  Multiply ALT26_POP by\t1e-05 because the largest (abs) value is\t301019.0\n[16:31:56] < Detailed >  Multiply ALT35_HHINC by\t1e-05 because the largest (abs) value is\t181965.0\n[16:31:56] < Detailed >  Multiply ALT37_DENS by\t1e-05 because the largest (abs) value is\t102846.33928097\n[16:31:56] < Detailed >  Multiply ALT84_HURENT by\t0.001 because the largest (abs) value is\t2929.0\n[16:31:56] < Detailed >  Multiply ALT60_POP by\t1e-05 because the largest (abs) value is\t301019.0\n[16:31:56] < Detailed >  Multiply ALT65_DENS by\t1e-05 becaus

In [83]:
# Associate utility functions with the numbering of alternatives (corresponds to the CHOSEN field created earlier)
V = utilities

# Associate the availability conditions with the alternatives
# for this model, all migrants had all alternatives theoretically available so all are equal to 1 (available)
# if individual people had different availability for alterantives, could pass in a column of the dataframe to account for that availability
av = {}
for i in range(0, 101):
    av[i] = 1

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, None, CHOSEN)


# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'us_full_basic'

# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[22:25:10] < General >   Remove 1868 unused variables from the database as only 308 are used.
[22:25:11] < Detailed >  It is suggested to scale the following variables.
[22:25:11] < Detailed >  Multiply ALT2_POP by	1e-05 because the largest (abs) value is	301019.0
[22:25:11] < Detailed >  Multiply ALT67_DIST by	1e-07 because the largest (abs) value is	5663184.0
[22:25:11] < Detailed >  Multiply ALT2_DENS by	1e-05 because the largest (abs) value is	102846.33928097
[22:25:11] < Detailed >  Multiply ALT26_POP by	1e-05 because the largest (abs) value is	301019.0
[22:25:11] < Detailed >  Multiply ALT37_DENS by	1e-05 because the largest (abs) value is	102846.33928097
[22:25:11] < Detailed >  Multiply ALT43_DENS by	1e-05 because the largest (abs) value is	102846.33928097
[22:25:11] < Detailed >  Multiply ALT60_POP by	1e-05 because the largest (abs) value is	301019.0
[22:25:11] < Detailed >  Multiply ALT65_DENS by	1e-05 because the largest (abs) value is	102846.33928097
[22:25:11] < Detailed >

KeyboardInterrupt: 